# 

- **Questions**: [Here](../data/project/mini_project.pdf)
- **Answer Set** : Project
- **Full Name** : Fatemeh Karimi Barikarasfi
- **Student Code** : 610301060

The goal of this projet is sequence classification using Bert model for Part of Speech Tagging and Named Entity Recognition problem at the same time.

[Dataset](https://drive.google.com/drive/folders/1YJvoIpInw0fYz2fjHSR65mjs6IYFZUvN)

## Importing Needed Libraries

In [3]:
import numpy as np
import pickle as pk
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score, precision_score, classification_report, accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

from transformers import BertTokenizerFast, TFBertModel, TFBertForTokenClassification
from keras.utils.np_utils import to_categorical

## Loading and Preprocessing and Preparing Data

In [4]:
with open('./drive/MyDrive/POS_NER/train.pickle', 'rb') as f:
    x_train = pk.load(f)

ner_set = set([tag for row in x_train["ner_tags"] for tag in row])
pos_set = set([tag for row in x_train["pos_tags"] for tag in row])

ner_label2id = {val: idx for idx, val in enumerate(ner_set)}
ner_id2label = {idx: val for idx, val in enumerate(ner_set)}

pos_label2id = {val: idx for idx, val in enumerate(pos_set)}
pos_id2label = {idx: val for idx, val in enumerate(pos_set)}

ner_max = len(ner_set)
pos_max = len(pos_set)

In [5]:
max([len(x) for x in x_train['tokens']])

124

In [6]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def generator(subset=0, batch_size=20):

    path = './drive/MyDrive/POS_NER/test.pickle'
    if subset == 0:
        path = './drive/MyDrive/POS_NER/train.pickle'
    if subset == 1:
        path = './drive/MyDrive/POS_NER/validation.pickle'

    with open(path, 'rb') as f:
        data = pk.load(f)

    def tags_map(tag, idx, word_ids, label2id):
        result = []
        last_id = None
        for id in word_ids:
            if (id == None) or (id == last_id) or (id >= len(data[tag][idx])):
                result.append(-100)
                continue

            last_id = id
            result.append(label2id[data[tag][idx][id]])

        return result

    for idxs in batch(range(len(data["tokens"])), batch_size):
        inputs = tokenizer(
            [" ".join(tokens) for tokens in data["tokens"][idxs]],
            add_special_tokens=True,
            return_tensors="tf",
            truncation=True,
            padding="max_length",
            max_length=128
        )

        if subset == 2:
            yield dict(inputs)
            continue

        yield dict(inputs), (
            tf.constant([tags_map("ner_tags", idx, inputs.word_ids(idx - idxs[0]), ner_label2id) for idx in idxs]),
            tf.constant([tags_map("pos_tags", idx, inputs.word_ids(idx - idxs[0]), pos_label2id) for idx in idxs])
        )

trainset = tf.data.Dataset.from_generator(generator, args=(0, 64), output_signature=(
    {
        "input_ids": tf.TensorSpec(shape=(None, 128), dtype=tf.int32),
        "token_type_ids": tf.TensorSpec(shape=(None, 128), dtype=tf.int32),
        "attention_mask": tf.TensorSpec(shape=(None, 128), dtype=tf.int32)
    },
    (
        tf.TensorSpec(shape=(None, 128), dtype=tf.int32),
        tf.TensorSpec(shape=(None, 128), dtype=tf.int32)
    )
))

validset = tf.data.Dataset.from_generator(generator, args=(1, 64), output_signature=(
    {
        "input_ids": tf.TensorSpec(shape=(None, 128), dtype=tf.int32),
        "token_type_ids": tf.TensorSpec(shape=(None, 128), dtype=tf.int32),
        "attention_mask": tf.TensorSpec(shape=(None, 128), dtype=tf.int32)
    },
    (
        tf.TensorSpec(shape=(None, 128), dtype=tf.int32),
        tf.TensorSpec(shape=(None, 128), dtype=tf.int32)
    )
))

testset = tf.data.Dataset.from_generator(generator, args=(2, 64), output_signature={
    "input_ids": tf.TensorSpec(shape=(None, 128), dtype=tf.int32),
    "token_type_ids": tf.TensorSpec(shape=(None, 128), dtype=tf.int32),
    "attention_mask": tf.TensorSpec(shape=(None, 128), dtype=tf.int32)
})

## Model

In [7]:
class BERTEncoder(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

        self.model = TFBertModel.from_pretrained("bert-base-uncased", trainable=True)

    def call(self, x):
        return self.model(x)[0]

In [8]:
ids = tf.keras.Input(shape=(128), name="input_ids", dtype=tf.int32)
types = tf.keras.Input(shape=(128), name="token_type_ids", dtype=tf.int32)
mask = tf.keras.Input(shape=(128), name="attention_mask", dtype=tf.int32)

output = BERTEncoder()({"input_ids": ids, "token_type_ids": types, "attention_mask": mask})

output_ner = tf.keras.layers.Dropout(0.1)(output)
output_ner = tf.keras.layers.Dense(ner_max, activation="softmax", name="ner", kernel_initializer=tf.keras.initializers.GlorotUniform())(output_ner)

output_pos = tf.keras.layers.Dropout(0.1)(output)
output_pos = tf.keras.layers.Dense(pos_max, activation="softmax", name="pos", kernel_initializer=tf.keras.initializers.GlorotUniform())(output_pos)


model = tf.keras.Model(inputs=[ids, types, mask], outputs=[output_ner, output_pos])

def accuracy(y_true, y_pred):
    mask = tf.not_equal(y_true, -100)
    y_true_filtered = tf.boolean_mask(y_true, mask)
    y_pred_filtered = tf.boolean_mask(y_pred, mask)
    accuracy = tf.keras.metrics.sparse_categorical_accuracy(y_true_filtered, y_pred_filtered)
    return tf.reduce_mean(accuracy)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics={
        "ner": [accuracy],
        "pos": [accuracy],
    },
    loss={
        "ner": tf.keras.losses.SparseCategoricalCrossentropy(ignore_class=-100),
        "pos": tf.keras.losses.SparseCategoricalCrossentropy(ignore_class=-100),
    },
)

model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert_encoder (BERTEncoder)     (None, 128, 768)     109482240   ['attention_mask[0][0]',         
                                                                  'input_ids[0][0]',          

In [9]:
model.fit(trainset, validation_data=validset, epochs=10, batch_size=64)

Epoch 1/10


220/220 [==============================] - 418s 2s/step - loss: 1.3986 - ner_loss: 0.2776 - pos_loss: 1.1210 - ner_accuracy: 0.9220 - pos_accuracy: 0.6993 - val_loss: 0.7723 - val_ner_loss: 0.1515 - val_pos_loss: 0.6208 - val_ner_accuracy: 0.9533 - val_pos_accuracy: 0.8140
Epoch 2/10
220/220 [==============================] - 364s 2s/step - loss: 0.6081 - ner_loss: 0.1132 - pos_loss: 0.4949 - ner_accuracy: 0.9644 - pos_accuracy: 0.8479 - val_loss: 0.5519 - val_ner_loss: 0.1113 - val_pos_loss: 0.4406 - val_ner_accuracy: 0.9654 - val_pos_accuracy: 0.8670
Epoch 3/10
220/220 [==============================] - 364s 2s/step - loss: 0.4253 - ner_loss: 0.0738 - pos_loss: 0.3515 - ner_accuracy: 0.9762 - pos_accuracy: 0.8889 - val_loss: 0.4968 - val_ner_loss: 0.1080 - val_pos_loss: 0.3888 - val_ner_accuracy: 0.9679 - val_pos_accuracy: 0.8822
Epoch 4/10
220/220 [==============================] - 364s 2s/step - loss: 0.3525 - ner_loss: 0.0635 - pos_loss: 0.2890 - ner_accuracy: 0.9789 - pos_accurac

## Model Evaluation

In [10]:
predicts = model.predict(validset)

51/51 [==============================] - 32s 565ms/step


In [11]:
ner_tags_true = tf.reshape(tf.concat([y[0] for x, y in validset], axis=0), [-1])
pos_tags_true = tf.reshape(tf.concat([y[1] for x, y in validset], axis=0), [-1])

ner_tags_pred = tf.reshape(predicts[0].argmax(-1), [-1])
pos_tags_pred = tf.reshape(predicts[1].argmax(-1), [-1])

ner_ignores = tf.where(tf.not_equal(ner_tags_true, -100))
pos_ignores = tf.where(tf.not_equal(pos_tags_true, -100))

In [12]:
ner_tags_true = tf.gather(ner_tags_true, ner_ignores)
pos_tags_true = tf.gather(pos_tags_true, pos_ignores)

ner_tags_pred = tf.gather(ner_tags_pred, ner_ignores)
pos_tags_pred = tf.gather(pos_tags_pred, pos_ignores)

Results of model for each tasks:

NER:

In [13]:
print("NER Recall: {0:.4f}".format(recall_score(ner_tags_true, ner_tags_pred, average='micro')))
print("NER Accuracy: {0:.4f}".format(accuracy_score(ner_tags_true, ner_tags_pred)))
print("NER Precision: {0:.4f}".format(precision_score(ner_tags_true, ner_tags_pred, average='micro')))
print('NER F1-macro score: {0:.4f}'.format(f1_score(ner_tags_true, ner_tags_pred, average='micro')))

NER Recall: 0.9763
NER Accuracy: 0.9763
NER Precision: 0.9763
NER F1-macro score: 0.9763


In [14]:
print(classification_report(ner_tags_true, ner_tags_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9875    0.9923    0.9899     39888
           1     0.9359    0.9492    0.9425      1614
           2     0.9501    0.9526    0.9514      1119
           3     0.9178    0.8792    0.8981      1498
           4     0.9092    0.8154    0.8598       921
           5     0.9183    0.9429    0.9304      1681
           6     0.9333    0.8376    0.8829       234
           7     0.8922    0.8317    0.8609       796
           8     0.8099    0.7259    0.7656       270

    accuracy                         0.9763     48021
   macro avg     0.9171    0.8808    0.8979     48021
weighted avg     0.9759    0.9763    0.9760     48021



POS:

In [15]:
print("POS Recall: {0:.4f}".format(recall_score(pos_tags_true, pos_tags_pred, average='micro')))
print("POS Accuracy: {0:.4f}".format(accuracy_score(pos_tags_true, pos_tags_pred)))
print("POS Precision: {0:.4f}".format(precision_score(pos_tags_true, pos_tags_pred, average='micro')))
print('POS F1-macro score: {0:.4f}'.format(f1_score(pos_tags_true, pos_tags_pred, average='micro')))

POS Recall: 0.9116
POS Accuracy: 0.9116
POS Precision: 0.9116
POS F1-macro score: 0.9116


In [16]:
print(classification_report(pos_tags_true, pos_tags_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9793    0.9525    0.9657       547
           1     1.0000    0.7500    0.8571         4
           2     0.9221    0.7889    0.8503        90
           3     0.9567    0.9641    0.9604       641
           4     0.9556    0.9333    0.9443       645
           5     0.8921    0.9572    0.9235      1753
           6     0.9401    0.9537    0.9469      1728
           7     0.9363    0.9242    0.9302       541
           8     0.9308    0.9474    0.9390       837
           9     0.9520    0.9499    0.9510      4655
          10     0.9512    0.9632    0.9572      3238
          11     0.9730    0.8780    0.9231        41
          12     0.7931    0.5610    0.6571        41
          13     0.9325    0.9409    0.9367      4465
          14     0.8639    0.7901    0.8253      2796
          15     0.7196    0.7857    0.7512        98
          16     0.8507    0.8507    0.8507        67
          17     1.0000    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Making Prediction on Test Dataset

In [17]:
predicts = model.predict(testset)
with open('./drive/MyDrive/POS_NER/test.pickle', 'rb') as f:
    x_test = pk.load(f)
    x_test_preprocessed = tokenizer(
        [" ".join(tokens) for tokens in x_test["tokens"]],
        return_tensors="tf",
        truncation=True,
        padding="max_length",
        max_length=128
    )

54/54 [==============================] - 32s 525ms/step


In [18]:
data = {
    "tokens": [],
    "ner_tags": [],
    "pos_tags": []
}

ner_predicts = predicts[0].argmax(-1)
pos_predicts = predicts[1].argmax(-1)
for idx in range(len(x_test["tokens"])):
    ids = x_test_preprocessed.word_ids(idx)
    data["tokens"].append(x_test["tokens"][idx])
    data["ner_tags"].append([ner_predicts[idx][ids.index(i)] for i in range(len(x_test["tokens"][idx]))])
    data["pos_tags"].append([pos_predicts[idx][ids.index(i)] for i in range(len(x_test["tokens"][idx]))])

with open('./drive/MyDrive/POS_NER/test_pred.pickle', 'wb') as f:
    pk.dump(data, f, protocol=pk.HIGHEST_PROTOCOL)